In [ ]:
import cvxpy as cp
import numpy as np
import pandas as pd
import json
import datetime
import matplotlib.pyplot as plt
import os
from matplotlib.patches import Patch
parent_dir = os.path.dirname(os.getcwd())
path = parent_dir 
save_path = parent_dir + '/'
from optimization import *
import warnings
warnings.filterwarnings("ignore")
color_palette = ['#332288',  '#117733', '#44AA99', '#88CCEE', '#DDCC77', '#CC6677', '#AA4499', '#882255']

## Figure 5: boxplot of annual costs by cluster

In [ ]:
period_string = '2024-06-03_to_2025-06-01'

num_clusters = 15
cluster_labels = np.array(pd.read_csv('Data/Cluster_Results_Adaptive/cluster_labels_'+str(num_clusters)+'.csv', header=None).iloc[:,0])
vinids = np.array(pd.read_csv('Data/vinids.csv').iloc[:,1])

#make a dataframe of vinids and cluster labels
vinids_df = pd.DataFrame({'vinid': vinids, 'cluster': cluster_labels})
label_list = np.unique(vinids_df['cluster'])

In [ ]:
def get_box_data(vinids_df, results_path, period_string):    
#load results into matrices
    vinids_df = pd.DataFrame({'vinid': vinids, 'cluster': cluster_labels})
    vin_length = 749
    cost_mat_uncontrolled = []
    cost_mat_controlled = []
    cost_mat_v2g_home = []
    cost_mat_v2g_everywhere = []
    vin_not_found_idx = []

    for vin in np.arange(1,vin_length):
        try:
            cost_mat_uncontrolled.append( pd.read_csv(results_path + 'cost_uncontrolled_' + period_string + '_vin_' +str(vin) + '.csv').iloc[:,1])
            cost_mat_controlled.append( pd.read_csv(results_path + 'cost_no_v2g_' + period_string + '_vin_' +str(vin) + '.csv').iloc[:,1])
            cost_mat_v2g_home.append( pd.read_csv(results_path + 'cost_v2g_home_' + period_string + '_vin_' +str(vin) + '.csv').iloc[:,1])
            cost_mat_v2g_everywhere.append(pd.read_csv(results_path + 'cost_v2g_everywhere_' + period_string + '_vin_' +str(vin) + '.csv').iloc[:,1])

        except:
            vin_not_found_idx.append(np.where(vinids==vin)[0])

    cost_mat_uncontrolled = np.array(cost_mat_uncontrolled)
    cost_mat_controlled = np.array(cost_mat_controlled)
    cost_mat_v2g_home = np.array(cost_mat_v2g_home)
    cost_mat_v2g_everywhere = np.array(cost_mat_v2g_everywhere)

    for remove in vin_not_found_idx:
        try:
            vinids_df = vinids_df.drop(vinids_df[vinids_df['vinid'] == vinids[remove[0]]].index)
        except:
            pass
    vinids_df = vinids_df.reset_index(drop=True)

    #replace nan with previous value
    for vin in np.arange(cost_mat_controlled.shape[0]):
        for week in np.arange(cost_mat_controlled.shape[1]):
            if np.isnan(cost_mat_controlled[vin, week]):
                if week == 0:
                    cost_mat_controlled[vin, week] = 0
                else:
                    cost_mat_controlled[vin, week] = cost_mat_controlled[vin, week-1]
            if np.isnan(cost_mat_uncontrolled[vin, week]):
                if week == 0:
                    cost_mat_uncontrolled[vin, week] = 0
                else:
                    cost_mat_uncontrolled[vin, week] = cost_mat_uncontrolled[vin, week-1]
            if np.isnan(cost_mat_v2g_home[vin, week]):
                if week == 0:
                    cost_mat_v2g_home[vin, week] = 0
                else:
                    cost_mat_v2g_home[vin, week] = cost_mat_v2g_home[vin, week-1]
            if np.isnan(cost_mat_v2g_everywhere[vin, week]):
                if week == 0:
                    cost_mat_v2g_everywhere[vin, week] = 0
                else:
                    cost_mat_v2g_everywhere[vin, week] = cost_mat_v2g_everywhere[vin, week-1]

    #delete rows that sum up to zero from controlled
    zero_idx = np.all(cost_mat_controlled == 0, axis=1)
    cost_mat_controlled = cost_mat_controlled[~zero_idx]
    cost_mat_uncontrolled = cost_mat_uncontrolled[~zero_idx]
    cost_mat_v2g_home = cost_mat_v2g_home[~zero_idx]
    cost_mat_v2g_everywhere = cost_mat_v2g_everywhere[~zero_idx]

    # drop the rows where zero_idx is True from vinids_df
    vinids_df = vinids_df[~zero_idx]
    vinids_df = vinids_df.reset_index(drop=True)
    return vinids_df, cost_mat_uncontrolled, cost_mat_controlled, cost_mat_v2g_home, cost_mat_v2g_everywhere



In [ ]:
def plot_box(ax, box_data_home, box_data_all):
    ax.boxplot(box_data_home, positions=np.arange(1,num_clusters+1)-0.16, widths=0.3, patch_artist=True, boxprops=dict(facecolor=color_palette[3], color=color_palette[3]), medianprops=dict(color='black'), whiskerprops=dict(color='black'), capprops=dict(color='black'), showfliers=False,flierprops=dict(markerfacecolor=color_palette[2], marker='o'))
    ax.boxplot(box_data_all, positions=np.arange(1,num_clusters+1)+0.16, widths=0.3, patch_artist=True, boxprops=dict(facecolor=color_palette[4], color=color_palette[4]), medianprops=dict(color='black'), whiskerprops=dict(color='black'), capprops=dict(color='black'), showfliers=False,flierprops=dict(markerfacecolor=color_palette[3], marker='o'))
    ax.set_ylabel('Annual Costs [$1000]', fontsize=16)
    ax.set_xlabel('Cluster', fontsize=16)
    ax.set_xticks(np.arange(1,num_clusters+1))
    ax.set_xticklabels(np.arange(1,num_clusters+1), fontsize=14) 
    
    #add light gray horizontal line at y=0
    ax.axhline(0, color='lightgray', linestyle='--', linewidth=1)
    #set y limits
    min=-6000
    max=6000
    ax.set_ylim(min, max)           
    ax.set_yticks(ax.get_yticks(), labels= np.round(np.arange(min/1000, max/1000+1,2),2), fontsize=14) 

In [ ]:
fig, ax = plt.subplots(4,1,figsize=(13, 18))
plt.subplots_adjust(hspace=0.2)
plt.subplots_adjust(wspace=0.15)

legend_elements = [Patch(facecolor=color_palette[3], color=color_palette[3],
                            label= 'V2G Home'),
                    Patch(facecolor=color_palette[4], color=color_palette[4],
                            label= 'V2G Everywhere')]

ax = ax.flatten()
fig_label = ['a.', 'b.', 'c.', 'd.']
cluster_info_df = pd.read_csv('Data/Cluster_Results_Adaptive/cluster_info_'+str(num_clusters)+'.csv')

for i, circuit in enumerate(['cir_1', 'cir_2', 'cir_3', 'cir_4']):
    aging = 'batt_aging_0'
    results_path = 'Results/'+ circuit + '/' + aging + '/' + 'elrp_1_'
    vinids_df, cost_mat_uncontrolled, cost_mat_controlled, cost_mat_v2g_home, cost_mat_v2g_everywhere = get_box_data(vinids_df, results_path, period_string)
    box_data_home = []
    box_data_all = []
    for c in range(num_clusters):

        num_periods = cost_mat_uncontrolled.shape[1]

        c_label = cluster_info_df['Orig Cluster Label'].values[c]

        box_data_home.append( np.cumsum(cost_mat_v2g_home[vinids_df.cluster == c_label, :], axis=1)[:,-1])    
        box_data_all.append( np.cumsum(cost_mat_v2g_everywhere[vinids_df.cluster == c_label, :], axis=1)[:,-1])    


    plot_box(ax[i], box_data_home, box_data_all)
    ax[i].annotate('Circuit '+circuit[-1], xy=(0.07, .91), xycoords='axes fraction', ha='center', fontsize=16)
    ax[i].legend(handles=legend_elements, loc='upper right', fontsize=14)
    ax[i].annotate(fig_label[i], xy=(-.07, 1.05), xycoords='axes fraction', ha='center', fontsize=16)


plt.savefig(save_path + '/5_cost_clusters_box' + '.pdf', bbox_inches='tight')